In [ ]:
#Coursera Capstone Project
#The Battle of Neighborhoods (Week 1)

1. Introduction.

1.1. Background:
Ho Chi Minh city is the biggest city in Vietnam with population of 9 million (13 million in the metropolitan area) as of 2019. This city not only attracts people from other region of Vietnam but also foreigners around the world to come to Ho Chi Minh city to work and live. The city is developing recently and people's lives are also improved. Ho Chi Minh city (HCMC) has 19 districts which designated urban or suburban with 494km2 and 5 rural districts with 1601km2. (Wikipedia - access on 30/3/2020 - https://en.wikipedia.org/wiki/Ho_Chi_Minh_City).
District 1 has most of the city's administrative offices, consulates and large buildings. District 1 is the busiest district in the city with the highest standard of living. District 1 is considered the financial center of Saigon and Vietnam. It is the home of Ho Chi Minh City Stock Exchange and the headquarters of international banks such as HSBC, ANZ, Standard Chartered, Citi, JPMorgan Chase, Bank of China, Maybank and so on. District 1 is also known as the best shopping destination in Vietnam, with a high concentration of high-end shopping malls and brands.
Living cost in district 1 is higher than other places in HCMC for both eating and living. The price of real estate (house, shop, lease land, house and apartment for rent...) is high, the price of 1 dish in distric 1 is also higher than other places.

The main means of transportation in the city is motorbikes, buses, taxis and bicycles. Over the past few years, cars have become more popular. The increasing number of vehicles tends to clog the power grid and contribute to air pollution. The government has called motorcycles a reason for the congestion and has developed plans to reduce the number of motorcycles and improve public transport. Ho Chi Minh City Metro, a rapid transit network, is being built in stages. The first line is under construction and is expected to be completed by the end of 2021. This first line will connect Ben Thanh with Suoi Tien Park in District 9, with a warehouse in Long Binh. Planners expect the route to serve more than 160,000 passengers a day. A route between Ben Thanh and Tham Luong in district 12 has been approved by the government and a few are subject to ongoing feasibility studies.

1.2 Problem:
Consider you plan to come to HCMC and work in district 1, with the high living cost and inconvenient transportation (especially you are foreigner), choosing an apartment to live with the reasonable price is very important. This project will provide more information about house price and distance so that people can decide which place they should lend.

1.3 Interested Audience:
For people who want to live in HCMC and work in district 1 with below conditions:
    - Apartment for 2 adults and 1 child (if any).
    - Price per month: USD 800 - 1,000
    - Distance: <= 7km or 30 minutes by bike
    
2. Data description.

I will use Foursquare to identify the venues around the commercial building (let choose Sai Gon tower - 29 Le Duan, Ben Nghe, distric 1, HCMC, VN) to estimate the distance from office to their apartment. I also collect data for apartment price in some districts nearby from internet (Hozzing and other sources). From the apartment in the list, I also use Foursquare to illustrate the amenities and essential facilities nearby for users to decide which one is suitable for them.

3. Methodology.
    - Collect Inspection Data
    - Explore and Understand Data
    - Data preparation and preprocessing 
    - Modeling

In [1]:
#Upload required library
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [2]:
CLIENT_ID = 'B03VPZND1O0OLEZEI3XF1YQLYHXKT0IZR0IV23413USUMDCX' # your Foursquare ID
CLIENT_SECRET = 'FBRWA0J05VYCGEPSMEK4NNXEEYE4ZSCGDOA0QO211MOWMYY1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B03VPZND1O0OLEZEI3XF1YQLYHXKT0IZR0IV23413USUMDCX
CLIENT_SECRET:FBRWA0J05VYCGEPSMEK4NNXEEYE4ZSCGDOA0QO211MOWMYY1


In [3]:
address = '29 Le Duan, Ben Nghe, district 1, Ho Chi Minh city, VN'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

10.7827759 106.7013413


In [4]:
neighborhood_latitude=10.7827759
neighborhood_longitude=106.7013413

In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 7000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=B03VPZND1O0OLEZEI3XF1YQLYHXKT0IZR0IV23413USUMDCX&client_secret=FBRWA0J05VYCGEPSMEK4NNXEEYE4ZSCGDOA0QO211MOWMYY1&v=20180604&ll=10.7827759,106.7013413&radius=7000&limit=100'

In [17]:
# results display is hidden for report simplification 
results = requests.get(url).json()

In [18]:
#Function that extracts the category of the venue - borrow from the Foursquare lab
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
HCMnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
HCMnearby_venues =HCMnearby_venues.loc[:, filtered_columns]
# filter the category for each row
HCMnearby_venues['venue.categories'] = HCMnearby_venues.apply(get_category_type, axis=1)
# clean columns
HCMnearby_venues.columns = [col.split(".")[-1] for col in HCMnearby_venues.columns]

HCMnearby_venues.shape

(100, 4)

In [21]:
HCMnearby_venues.head(10)

,name,categories,lat,lng
0,Pizza 4P’s Hai Bà Trưng,Pizza Place,10.783017,106.697017
1,Heart Of Darkness,Bar,10.779781,106.702999
2,MGallery Hotel des Art,Hotel,10.782382,106.697103
3,Pizza 4P's,Pizza Place,10.781866,106.705113
4,Park Hyatt Saigon,Hotel,10.777574,106.703609
5,Miu Miu Spa,Spa,10.781715,106.704754
6,Social Club @ Hotel Des Arts,Hotel Bar,10.782372,106.697303
7,Đường Sách Nguyễn Văn Bình,Bookstore,10.780249,106.699404
8,"Hum Vegetarian, Lounge & Restaurant",Vegetarian / Vegan Restaurant,10.776684,106.705873
9,Quan Chi Hoa Vietnamese Cuisine,Vietnamese Restaurant,10.779048,106.703471


In [23]:
latitude=10.7827759 
longitude=106.7013413
# create map of Ho Chi Minh city - dist.,1 place using latitude and longitude values
map_HCM = folium.Map(location=[latitude, longitude], zoom_start=18)

# add markers to map
for lat, lng, label in zip(HCMnearby_venues['lat'], HCMnearby_venues['lng'], HCMnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_HCM)  
    
map_HCM